In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# all_runs

- run locations

In [3]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
current run,..\..\..\..
2050 new LU,\\modelstation1\c$\workspace\sc_2050-with_2023...
"2050 new LU, less WFH, no telecommute",\\modelstation1\c$\workspace\sc_2050_base_year...
2050 old LU,\\modelstation1\c$\workspace\sc_2050_23baseyea...
2050 RTP,L:\RTP_2022\final_runs\sc_rtp_2050_constrained...


## System Summary

In [4]:
# Load VMT, VHT, and delay
df_vmt = summary_data.load_agg_data('network/vmt_facility.csv')
df_vht = summary_data.load_agg_data('network/vht_facility.csv')
df_delay = summary_data.load_agg_data('network/delay_facility.csv')

df_vmt = summary_data.load_agg_data('network/vmt_facility.csv')
df_vmt = pd.DataFrame(df_vmt.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1), columns=['VMT'])
df_vht = summary_data.load_agg_data('network/vht_facility.csv')
df_vht = pd.DataFrame(df_vht.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1), columns=['VMT'])
df_delay = summary_data.load_agg_data('network/delay_facility.csv')
df_delay = pd.DataFrame(df_delay.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1), columns=['VMT'])

In [5]:
# network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
# network_summary = network_summary.rename(columns={'total_delay': 'Delay'})

# Load VMT, VHT, and delay
df_vmt = summary_data.load_agg_data('network/vmt_facility.csv')
df_vmt = df_vmt.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1)
df_vht = summary_data.load_agg_data('network/vht_facility.csv')
df_vht = df_vht.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1)
df_delay = summary_data.load_agg_data('network/delay_facility.csv')
df_delay = df_delay.groupby('source').sum()[['arterial','connector','highway']].sum(axis=1)

# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
#Light rail boardings
df_line_boardings = summary_data.load_agg_data('transit/transit_line_results.csv')
lr_boardings = df_line_boardings[df_line_boardings['mode']=='r']
lr_ridership = lr_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [6]:
run_summary2 = pd.DataFrame({
    'VMT': df_vmt,
    'VHT': df_vht,
    'Delay': df_delay,
    'Transit Boardings': transit_ridership,
    'Light Rail Boardings': lr_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
source,,,,,,,
2050 RTP,"106,132,676","3,547,251","619,306","2,333,391","783,454",6.2%,"37,682"
2050 new LU,"100,777,478","3,237,461","409,904","1,469,239","426,616",4.5%,"35,579"
"2050 new LU, less WFH, no telecommute","102,521,097","3,311,479","431,951","1,412,193","421,104",4.2%,"36,264"
2050 old LU,"101,943,341","3,283,812","425,883","1,447,979","436,816",4.5%,"36,078"
current run,"82,978,640","2,556,854","239,405","485,465","92,876",2.1%,"41,738"
